In [68]:
#! pip install -r requirements.txt

In [69]:
import pandas as pd
from requests_oauthlib import OAuth1Session
import json
import yaml

In [70]:
"""
settings.ymlに以下のように認証情報を設定

===
twitter_credentials:
  client_key : XXX
  client_secret : XXX
  access_token : XXXX
  access_token_secret : XXXX
===
"""

f = open("settings.yml", "r")
settings = yaml.load(f)
credentials = settings['twitter_credentials']


In [71]:
SEARCH_TWEETS_URL = 'https://api.twitter.com/1.1/search/tweets.json'
RATE_LIMIT_STATUS_URL = "https://api.twitter.com/1.1/application/rate_limit_status.json"
SEARCH_LIMIT_COUNT = 100


In [123]:
params = {'q' : 'モンスト', 'count' : 5}

In [124]:
url = "https://api.twitter.com/1.1/search/tweets.json"
twitter = OAuth1Session(credentials['client_key'],
                        credentials['client_secret'],
                        credentials['access_token'],
                        credentials['access_token_secret']
                       )

In [125]:
from time import sleep
from dateutil.parser import parse
 
columns = ['timestamp', 'user_id', 'tweet', 'favorite_count']
df = pd.DataFrame(columns=columns)

max_id = -1
MAX_LOOP_COUNT = 10

for i in range(MAX_LOOP_COUNT):
    if max_id != -1:
        params['max_id']  = int(max_id) - 1  
    req = twitter.get(url, params = params)
    
    # レートリミットの残りを定期表示
    limit = int(req.headers['x-rate-limit-remaining'])
    if limit % 5 == 0:
        print("remain limit: {}".format(limit))
    
    if req.status_code == 200:
        search_timeline = json.loads(req.text)
        
        if search_timeline['statuses'] == []:
            break
        
        for tweet in search_timeline['statuses']:
            created_at = parse(tweet['created_at'])
            user_id = tweet['user']['id']
            text = tweet['text']
            like = tweet['favorite_count']
            
            append_list = [created_at, user_id, text, like]
            
            df_next = pd.DataFrame(
                [append_list],
                columns=columns
            )
            
            df = df.append(df_next)
        max_id = search_timeline['statuses'][-1]['id']
    
    else: 
        reset = req.headers['x-rate-limit-reset'] #リクエスト叶残数リセットまでの時間(UTC)
        sec = int(req.headers['X-Rate-Limit-Reset']) - time.mktime(datetime.datetime.now().timetuple()) #UTCを秒数に変換

        print ("""
        reset utc time: {}
        reset sec: {}
        """.format(reset, sec))
        sleep(sec)

remain limit: 125
remain limit: 120


In [126]:
df = df.reset_index(drop=True)

In [129]:
#df

In [131]:
DATA_DIR = "data/"
file_name = datetime.datetime.now().strftime("%Y%m%d_%H%M") + ".csv"
df.to_csv(DATA_DIR + file_name)